In [13]:
# https://github.com/llSourcell/pytorch_in_5_minutes/blob/master/demo.py

import torch
from torch.autograd import Variable
import pandas as pd
from random import randint
from ast import literal_eval
import numpy as np
import torch.nn as nn
# dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [5]:
#read in relevant data
trainData = np.loadtxt('data/trainData.txt', dtype=np.float32)
queryData = np.loadtxt('data/queryData.txt', dtype=np.float32)
df =  pd.read_pickle("./data/KNN.pkl")
querySize = len(queryData)

In [19]:
# BATCH_SIZE is batch size; INPUT_D is input dimension; HIDDEN_D is hidden dimension; 
BATCH_SIZE, INPUT_D, HIDDEN_D = 1000, 192, 128
ALPHA = 5
LEARNING_RATE = 1e-4
K = 5

# Create random Tensor for trainable features, and wrap them in Variables.
# requires_grad=True indicates that we want to compute gradients wrt these Variables during the backward pass.
anchors = Variable(torch.randn(HIDDEN_D, INPUT_D).type(torch.FloatTensor), requires_grad=True)
biases = Variable(torch.ones(HIDDEN_D).type(torch.FloatTensor) + 16, requires_grad=True)

In [16]:
def generateTripplet(index):
#     point = torch.rand(3)
#     pos = torch.rand(3)
#     neg = torch.rand(3)
    point = torch.tensor(queryData[index])
    pos = torch.tensor(trainData[df.iloc[index].KNN[randint(0,K)]])
    neg = torch.tensor(trainData[df.iloc[index].KNN[randint(K, len(df)-1)]])
    return point, pos, neg

In [17]:
def forward_pass(query):
    return torch.sign(torch.norm(query - anchors, 2, 1) - biases)

In [21]:
#training
for epoch in range(100):
    
    #generate batch and compute collective loss for batch
    batch_indicies = np.random.choice(querySize, BATCH_SIZE, replace=False)
    loss = 0
    for index in batch_indicies:
        query, pos, neg = generateTripplet(index)
        queryMapped, posMapped, negMapped = [forward_pass(x) for x in [query, pos, neg]]
        pos_neg_diff = torch.norm(queryMapped-posMapped, 2, 0).pow(2) - torch.norm(queryMapped-negMapped, 2, 0).pow(2) + ALPHA
        loss += nn.functional.relu(pos_neg_diff).sum()
    
    print(epoch, loss.data)

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Variables with requires_grad=True.
    # After this we can call var.grad on variables
    loss.backward()

    # Update weights using gradient descent
    biases.data -= LEARNING_RATE * biases.grad.data
    anchors.data -= LEARNING_RATE * anchors.grad.data

    # Manually zero the gradients 
    biases.grad.data.zero_()
    anchors.grad.data.zero_()

0 tensor(16205.)
1 tensor(16062.)
2 tensor(16682.)
3 tensor(14930.)
4 tensor(19375.)
5 tensor(18750.)
6 tensor(16363.)
7 tensor(17525.)
8 tensor(16168.)
9 tensor(14161.)
10 tensor(16416.)
11 tensor(18216.)
12 tensor(17233.)
13 tensor(20717.)
14 tensor(18552.)
15 tensor(17681.)
16 tensor(17264.)
17 tensor(16092.)
18 tensor(19275.)
19 tensor(16747.)
20 tensor(15122.)
21 tensor(16973.)
22 tensor(18847.)
23 tensor(16342.)
24 tensor(14494.)
25 tensor(16245.)
26 tensor(17018.)
27 tensor(16296.)
28 tensor(16560.)
29 tensor(20323.)
30 tensor(19743.)
31 tensor(14539.)
32 tensor(17582.)
33 tensor(18760.)
34 tensor(17231.)
35 tensor(20009.)
36 tensor(15314.)
37 tensor(17935.)
38 tensor(15302.)
39 tensor(15526.)
40 tensor(14381.)
41 tensor(16811.)
42 tensor(15171.)
43 tensor(18854.)
44 tensor(14712.)
45 tensor(18155.)
46 tensor(16047.)
47 tensor(15388.)
48 tensor(18205.)
49 tensor(18246.)
50 tensor(14443.)
51 tensor(16626.)
52 tensor(16999.)
53 tensor(14039.)
54 tensor(16066.)
55 tensor(17060.)
56

In [ ]:
t0 = torch.tensor([1.,2.,3.])
t1 = torch.tensor([[1.,1.,1.], [2.,2.,2.]])
t2 = torch.tensor([[2.,2.,2.], [3.,3.,3.]], requires_grad = True)
# t3 = torch.sign(torch.norm(t1 - t2, 2, 1) + biases[0:2])

In [ ]:
t2 - t1